In [1]:
!pip install pyspark

In [17]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract
from pyspark.sql.functions import to_date, date_format


In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
videos_stats = spark.read.csv("/content/drive/MyDrive/atividadeTratamento/videos_stats.csv", header=True, inferSchema=True)

In [6]:
print('dataframe videos_stats: \n', videos_stats.show(5))

+---+--------------------+-----------+------------+-------+-------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech|76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech|63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech|71566.0|  1426.0| 922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech|96513.0|  5155.0|1855644.0|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
only showing top 5 rows

dataframe videos_stats: 
 None


In [7]:
colunas = ['Likes', 'Views', 'Comments']

for coluna in colunas:
    videos_stats = videos_stats.na.fill({coluna: 0})

print()

In [8]:
comments = spark.read.csv("/content/drive/MyDrive/atividadeTratamento/comments.csv", header=True, inferSchema=True)

In [9]:
print('dataframe comments: \n', comments.show(n=5, truncate=False))

+---+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------+
|_c0|Video ID   |Comment                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |Likes|Sentiment|
+---+-------

In [ ]:
print('total registros: ', videos_stats.count())
print('total comentarios', comments.count())
print('total geral', comments.count() + videos_stats.count())

total registros:  1881
total comentarios 30036
total geral 31917


In [10]:
pedidos_remocao_nulos = videos_stats.na.drop()
print('Total de videos após limpeza de valores nulos: ', pedidos_remocao_nulos.count())

pedidos_remocao_nulos_id = comments.na.drop(subset=['video ID'])
print('Total de videos após limpeza de id nulos: ', pedidos_remocao_nulos_id.count())

Total de videos após limpeza de valores nulos:  1881
Total de videos após limpeza de id nulos:  22555


In [11]:
videos_stats.groupBy("Video ID").count().filter(col("count") > 1).show()

+-----------+-----+
|   Video ID|count|
+-----------+-----+
|7eh4d6sabA0|    2|
|4mgePWWCAmA|    2|
|5q87K1WaoFI|    2|
|R-R0KrXvWbc|    2|
|WBK2_lD7KGA|    2|
|RFMi3v0TXP8|    2|
|96mrgd8-3yE|    2|
|mqc6QqoGNWI|    2|
|2FYvHn12pOQ|    2|
|kkOweffr3II|    2|
|LeC5yJq4tsI|    2|
|ZgeorpjGJC0|    2|
+-----------+-----+



In [12]:
convertidosint_tratados_df = videos_stats.\
    withColumn('Likes', col('Likes').cast('int')).\
    withColumn('comments', col('comments').cast('int')).\
    withColumn('views', col('views').cast('int'))

convertidosint_tratados_df.show()
convertidosint_tratados_df.printSchema()

+---+--------------------+-----------+------------+-------+------+--------+--------+
|_c0|               Title|   Video ID|Published At|Keyword| Likes|comments|   views|
+---+--------------------+-----------+------------+-------+------+--------+--------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407|     672|  135612|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779|    4306| 1758063|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825|    3338| 1564007|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566|    1426|  922918|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513|    5155| 1855644|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570|    1643|  943119|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047|    9367| 5937790|
|  7|I bought the STRA...|2TqOmtTAMRY|  2022-08-07|   tech|216935|   12605| 4782514|
|  8|15 Emerging Techn...|wLlL46pYcg4|  2021-12-08|   tech| 45565

In [13]:
alteracoesComment_df = comments.\
    withColumn('Likes', col('Likes').cast('int')).\
    withColumn('Sentiment', col('Sentiment').cast('int')).\
    withColumnRenamed("Likes", "Likes Comment")


alteracoesComment_df.show()
alteracoesComment_df.printSchema()

+--------------+-----------+--------------------+-------------+---------+
|           _c0|   Video ID|             Comment|Likes Comment|Sentiment|
+--------------+-----------+--------------------+-------------+---------+
|             0|wAZZ-UWGVHI|Let's not forget ...|           95|        1|
|             1|wAZZ-UWGVHI|Here in NZ 50% of...|           19|        0|
|             2|wAZZ-UWGVHI|I will forever ac...|          161|        2|
|             3|wAZZ-UWGVHI|Whenever I go to ...|            8|        0|
|             4|wAZZ-UWGVHI|Apple Pay is so c...|           34|        2|
|             5|wAZZ-UWGVHI|We’ve been houndi...|            8|        1|
|             6|wAZZ-UWGVHI|We only got Apple...|           29|        2|
|             7|wAZZ-UWGVHI|For now, I need b...|            7|        1|
|             8|wAZZ-UWGVHI|In the United Sta...|            2|        2|
|             9|wAZZ-UWGVHI|In Cambodia, we h...|           28|        1|
|            10|b3x28s61q3c|Wow, you r

In [14]:
campo_interaction_df = videos_stats.\
    withColumn('Likes', col('Likes').cast('float')).\
    withColumn('comments', col('comments').cast('float')).\
     withColumn('views', col('views').cast('float')).\
    withColumn('interaction', col('Likes') + col('Comments') + col('Views'))

campo_interaction_df.show()

+---+--------------------+-----------+------------+-------+--------+--------+-----------+-----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|comments|      views|interaction|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+-----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|   139691.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|  1839148.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|  1631170.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|   995910.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|  1957312.0|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|   978332.0|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|  593

In [21]:
videos_stats_df = videos_stats.withColumn("data", to_date(col("Published At")))

videos_stats_df.show()
videos_stats_df.printSchema()

+---+--------------------+-----------+------------+-------+--------+--------+-----------+----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|      data|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|2022-08-23|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|2022-08-24|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|2022-08-23|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|2022-08-23|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|2022-08-08|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|2021-11-05|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|  5937790.0|20

In [26]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.functions import year

videos_stats_df = videos_stats_df.withColumn("Publish At", year("data"))

videos_stats_df.show()

+---+--------------------+-----------+------------+-------+--------+--------+-----------+----------+----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|      data|Publish At|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+----------+----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|2022-08-23|      2022|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|2022-08-24|      2022|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|2022-08-23|      2022|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|2022-08-23|      2022|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|2022-08-08|      2022|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|2021-11-05|    

In [28]:
join_videos_stats_df = comments.join(videos_stats, 'Video ID', 'inner')

join_videos_stats_df.show()

+-----------+---+--------------------+------+---------+---+--------------------+------------+-------+-------+--------+---------+
|   Video ID|_c0|             Comment| Likes|Sentiment|_c0|               Title|Published At|Keyword|  Likes|Comments|    Views|
+-----------+---+--------------------+------+---------+---+--------------------+------------+-------+-------+--------+---------+
|wAZZ-UWGVHI|  0|Let's not forget ...|  95.0|      1.0|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|wAZZ-UWGVHI|  1|Here in NZ 50% of...|  19.0|      0.0|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|wAZZ-UWGVHI|  2|I will forever ac...| 161.0|      2.0|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|wAZZ-UWGVHI|  3|Whenever I go to ...|   8.0|      0.0|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|wAZZ-UWGVHI|  4|Apple Pay is so c...|  34.0|      2.0|  0|Apple Pay Is Kill...|  2022-08-23|   t

In [29]:
USvideos = spark.read.csv("/content/drive/MyDrive/atividadeTratamento/USvideos.csv", header=True, inferSchema=True)

In [30]:
print('dataframe USvideos: \n', USvideos.show(5))


+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [31]:
join_USvideos_df = USvideos.join(videos_stats, 'title', 'inner')

join_USvideos_df.show()

+--------------------+-----------+-------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+----+-----------+------------+----------------+--------+--------+------------+
|               title|   video_id|trending_date|    channel_title|category_id|        publish_time|                tags|   views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description| _c0|   Video ID|Published At|         Keyword|   Likes|Comments|       Views|
+--------------------+-----------+-------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+----+-----------+------------+----------------+--------+--------+

In [35]:
for coluna in videos_stats.columns:
    videos_stats.select(count(when(col(coluna).isNull(), coluna)).alias(coluna)).show()


+---+
|_c0|
+---+
|  0|
+---+

+-----+
|Title|
+-----+
|    0|
+-----+

+--------+
|Video ID|
+--------+
|       0|
+--------+

+------------+
|Published At|
+------------+
|           0|
+------------+

+-------+
|Keyword|
+-------+
|      0|
+-------+

+-----+
|Likes|
+-----+
|    0|
+-----+

+--------+
|Comments|
+--------+
|       0|
+--------+

+-----+
|Views|
+-----+
|    0|
+-----+



In [48]:
colunas = videos_stats_df.columns
print(colunas)

videos_stats_df = videos_stats_df.drop('_c0')

videos_stats_df.show()


['Title', 'Video ID', 'Published At', 'Keyword', 'Likes', 'Comments', 'Views', 'data', 'Publish At']
+--------------------+-----------+------------+-------+--------+--------+-----------+----------+----------+
|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|      data|Publish At|
+--------------------+-----------+------------+-------+--------+--------+-----------+----------+----------+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|2022-08-23|      2022|
|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|2022-08-24|      2022|
|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|2022-08-23|      2022|
|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|2022-08-23|      2022|
|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|2022-08-08|      2022|
|Brewmaster Answer...|18fwz9Itbvo| 

In [49]:
videos_stats_df.write.mode('overwrite').option('header', 'true').parquet('drive/MyDrive/Colab Notebooks/spark/data/output/videos-tratados-parquet')

In [50]:
colunas = join_USvideos_df.columns
print(colunas)

join_USvideos_df = join_USvideos_df.drop('_c0')

join_USvideos_df.show()

['title', 'video_id', 'trending_date', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description', '_c0', 'Video ID', 'Published At', 'Keyword', 'Likes', 'Comments', 'Views']
+--------------------+-----------+-------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-----------+------------+----------------+--------+--------+------------+
|               title|   video_id|trending_date|    channel_title|category_id|        publish_time|                tags|   views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|   Video ID|Published At|         Keyword|   Likes|Comments|       Views|
+---------------

In [ ]:
join_USvideos_df.write.mode('overwrite').option('header', 'true').parquet('drive/MyDrive/Colab Notebooks/spark/data/output/videos-comments-tratados-parquet')